# Blockchain Testing

This notebook is adapted from a series of unit tests developed to test the network. If you would prefer to see these tests in a more traditional manner (using pytest) see the [repository here](https://github.com/Gamma749/Hypderledger-Iroha-Multinode-Demo)

---

First we will import the neccesarry modules and set a new domain. Because these commits are non-unique (unlike hashes) then we need to ensure they are sent to a different domain each time.

In [ ]:
from IrohaUtils import *
import socket
import random

def printColor(message, color=bcolors.OKGREEN):
    print(f"{bcolors.OKGREEN}{message}{bcolors.ENDC}")

# Set the IP addresses of the nodes, for easy access later
NODE_LOCATIONS = [
        (IROHA_HOST_ADDR_1, int(IROHA_PORT_1)),
        (IROHA_HOST_ADDR_2, int(IROHA_PORT_2)),
        (IROHA_HOST_ADDR_3, int(IROHA_PORT_3)),
        (IROHA_HOST_ADDR_4, int(IROHA_PORT_4)),
    ]

We need to create user accounts with roles. A role in an Iroha context is like a security group, allowing users to only perform actions according to the permissions of their roles. We want our users to be able to transfer and receive assets, so we grant the role these permissions. Let test if we can create roles:

In [ ]:
# In case this cell is run multiple times, create a unique role name
ROLE_NAME = None
while True:
    ROLE_NAME = f"user_{random.randint(1,1000000)}"
    print(f"Attempting {ROLE_NAME=}")
    commands = [
        admin["iroha"].command("CreateRole", role_name=ROLE_NAME, permissions=[
            primitive_pb2.can_receive,
            primitive_pb2.can_transfer,
            primitive_pb2.can_grant_can_transfer_my_assets
        ])
    ]
    tx = IrohaCrypto.sign_transaction(
        admin["iroha"].transaction(commands), ADMIN_PRIVATE_KEY)
    # Send the transaction and get the status
    status = send_transaction(tx, net_1)
    if status[0] == "COMMITTED": break
printColor(status)
printColor(f"Role name for tests is {ROLE_NAME}")

We will also need a new domain to execute these tests in. In Iroha, a domain is somewhat like a namespace; all transactions within a domain affect only the assets within that domain (and do not spill to other domains). This means each time we create a new domain for these tests we can run them fresh using domains. Let us try and make a domain:

In [ ]:
DOMAIN_NAME = None
while True:
    # Set domain name to some random digit
    DOMAIN_NAME = f"domain-{random.randint(1,1000000)}"
    print(f"Trying {DOMAIN_NAME=}")
    # Create the domain by sending the request to Iroha
    commands = [
        admin["iroha"].command('CreateDomain', domain_id=DOMAIN_NAME, default_role=ROLE_NAME)
    ]
    tx = IrohaCrypto.sign_transaction(
        admin["iroha"].transaction(commands), admin["private_key"])
    # Send the transaction and get the status
    status = send_transaction(tx, net_1)
    print(status)
    if status[0] == "COMMITTED":
        print("SUCCESSFULLY CREATED DOMAIN")
        break

printColor(f"Your domain is {DOMAIN_NAME}")

We will need an asset to pass around during these test. An Iroha asset is like a currency, in the sense that it can be transferred between users. However, during this project we have used assets for many other purposes, and I feel it neccessary to emphasise that blockchain technology is more than cryptocurrency. 

In [ ]:
# We do not need to create a randomly named asset, since the domain is already unique
commands = admin["iroha"].transaction([
        admin["iroha"].command('CreateAsset', asset_name="asset",
                domain_id=DOMAIN_NAME, precision=2)
    ])
tx = IrohaCrypto.sign_transaction(commands, admin["private_key"])
status = send_transaction(tx, net_1, verbose=True)
print(status)
if status[0] == "COMMITTED":
    printColor("ASSET CREATED")
ASSET_ID = f"asset#{DOMAIN_NAME}"

In [ ]:
# Add 1000 asset to the admin account
commands = admin["iroha"].transaction([
        admin["iroha"].command('AddAssetQuantity',
                      asset_id=ASSET_ID, amount='1000.00')
    ])
tx = IrohaCrypto.sign_transaction(commands, admin["private_key"])
status = send_transaction(tx, net_1, verbose=True)
print(status)
if status[0] == "COMMITTED":
    printColor("ASSET ADDED TO ADMIN")

Finally, we can look into making some users! An account needs no extra introduction, it is a users interface and credentials with the Iroha network.

In [ ]:
# Use the IrohaUtils new_user method to generate keys etc, stored in a dict
user_a = new_user("user_a", DOMAIN_NAME)
user_b = new_user("user_b", DOMAIN_NAME)
user_c = new_user("user_c", DOMAIN_NAME)

commands = [
        # Create users a,b,c
        admin["iroha"].command('CreateAccount', account_name=user_a["name"], domain_id=user_a["domain"],
                          public_key=user_a["public_key"]),
        admin["iroha"].command('CreateAccount', account_name=user_b["name"], domain_id=user_b["domain"],
                          public_key=user_b["public_key"]),
        admin["iroha"].command('CreateAccount', account_name=user_c["name"], domain_id=user_c["domain"],
                          public_key=user_c["public_key"])
    ]
tx = IrohaCrypto.sign_transaction(
    iroha_admin.transaction(commands), admin["private_key"])
status = send_transaction(tx, net_1)
if status[0] == "COMMITTED":
    printColor("USERS CREATED")

Let's give the users each some asset, so we can test with it:

In [ ]:
commands = [
    # Create users a,b,c
    admin["iroha"].command('TransferAsset', src_account_id=admin["id"], dest_account_id=user_a["id"],
                      asset_id=ASSET_ID, amount="100"),
    admin["iroha"].command('TransferAsset', src_account_id=admin["id"], dest_account_id=user_b["id"],
                      asset_id=ASSET_ID, amount="100"),
    admin["iroha"].command('TransferAsset', src_account_id=admin["id"], dest_account_id=user_c["id"],
                      asset_id=ASSET_ID, amount="100")
]
tx = IrohaCrypto.sign_transaction(
    iroha_admin.transaction(commands), ADMIN_PRIVATE_KEY)
status = send_transaction(tx, net_1)
assert status[0] == "COMMITTED"
printColor("TRANSFERAL OF ASSETS COMPLETE")

In the following tests we will be transferring assets back and forth a lot, and it would be very convinient to reset the asset amount between tests. Luckily, we can grant the admin account the ability to do this reset for us, so we will set this up now:

In [ ]:
 for user in [user_a, user_b, user_c]:
    print(f"{user['id']} GRANTS ADMIN TRANSFER PERMISSION")
    tx = user["iroha"].transaction([
        user["iroha"].command("GrantPermission", account_id=admin["id"], permission=primitive_pb2.can_transfer_my_assets)
    ], creator_account=user["id"])
    tx = IrohaCrypto.sign_transaction(
        tx, user["private_key"])
    status = send_transaction(tx, net_1)
    assert status[0] == "COMMITTED"
    print(f"{user['id']} SUCCESSFULLY GRANTED PERMISSION TO ADMIN")

printColor("All users granted admin permission to transfer assets")

In [ ]:
def get_user_assets(user_id):
    """
    Get all of the assets of a user and return these
    Args:
        user_id (string): The identity of the user to query, already on the blockchain
    Returns:
        List of account asset: The assets of the specified user
    """

    query = iroha_admin.query("GetAccountAssets", account_id=f"{user_id}")
    IrohaCrypto.sign_query(query, admin["private_key"])
    response = net_1.send_query(query)
    data = response.account_assets_response.account_assets
    return data

def set_user_asset_balance(users = [user_a, user_b, user_c], balance = 100):
    """
    Set the asset balance of all users to 100 for testing purposes
    """

    commands = []
    for user in users:
        user_assets = get_user_assets(user['id'])
        # Because there is only one asset, we can hardcode index 0
        user_coin_balance = user_assets[0].balance
        # Give admin 100, add 100 coins to user
        new_commands = [
            iroha_admin.command('AddAssetQuantity',
                      asset_id=ASSET_ID, amount=str(balance)),
            iroha_admin.command('TransferAsset', src_account_id=admin["id"], dest_account_id=user["id"],
                            asset_id=ASSET_ID, description="Top up coin", amount=str(balance))
        ]

        #Note the Iroha considers a movement of 0 coins stateless invalid, so lets handle this case
        if user_coin_balance != '0':
            # subtract balance from user
            new_commands.append(
            iroha_admin.command('TransferAsset', src_account_id=user["id"], dest_account_id=admin["id"],
                        asset_id=ASSET_ID, description="Transfer excess user balance back", amount=f"{user_coin_balance}")
            )
        for c in new_commands:
            commands.append(c)
        
    tx = IrohaCrypto.sign_transaction(
        admin["iroha"].transaction(commands), admin["private_key"])
    status = send_transaction(tx, net_1)
    assert status[0] == "COMMITTED"
    for user in [user_a, user_b, user_c]:
        user_assets = get_user_assets(user['id'])
        assert str(user_assets) == f'[asset_id: "asset#{DOMAIN_NAME}"\naccount_id: "{user["id"]}"\nbalance: "{balance}"\n]'
    printColor(f"Users balance reset to {balance}")

We can test this more complicated function now, let's set all account balances to 1000:

In [ ]:
set_user_asset_balance(balance=1000)
print("")
for user in [user_a, user_b, user_c]:
        user_assets = get_user_assets(user['id'])
        print(user_assets)

Excellent! We now have a domain with users and an asset we can transfer about at will. We can also easily reset this asset as needed. We have successfully set up a nice testing environment. In the process we have also tested that we could interact with the Iroha network, so a great deal of testing has already occured! At this point we may be interested in the blockchain state. We can easily look at all of the commited blocks with the following code, which you can run at any time to see the results of a test on the chain.

In [ ]:
log_all_blocks("testing.log")

This creates a file in `/logs` that stores our blockchain state in JSON format. If you look towards the end of the file, we can see some blocks with many transactions occuring, which corresponds to us setting the user balances in the previous cell! While the JSON format can be tedious to look through, it can provide some good insight as to what the blockchain is actually storing.

We are now ready to move on to the "real" testing.

---

Let's test if an honest transaction can take place. User B will send 10 coins to User C

In [ ]:
set_user_asset_balance()

command = [
    user_b["iroha"].command("TransferAsset", src_account_id=user_b["id"], dest_account_id=user_c["id"],
                        asset_id=ASSET_ID, amount="10")
]
tx = IrohaCrypto.sign_transaction(
    user_b["iroha"].transaction(command), user_b["private_key"])
status = send_transaction(tx, net_1)
assert status[0] == "COMMITTED"

# Now check that both parties have the correct asset total
user_b_assets = get_user_assets(user_b["id"])
user_c_assets = get_user_assets(user_c["id"])
assert str(user_b_assets) == f'[asset_id: "{ASSET_ID}"\naccount_id: "{user_b["id"]}"\nbalance: "90"\n]'
assert str(user_c_assets) == f'[asset_id: "{ASSET_ID}"\naccount_id: "{user_c["id"]}"\nbalance: "110"\n]'
printColor("HONEST TRANSFER COMPLETE")
print(user_b_assets)
print(user_c_assets)

Excellent! So two users can transfer assets between each other in an honest fashion (i.e. they have the assets to send)

We will now test if a user can "double spend". Can a user spend all of their asset on one transaction but spend that same asset on a *different* transaction before the blockchain realizes and stops it? User A will attempt a double spend to User B and User C

In [ ]:
set_user_asset_balance()

command = [
    user_a["iroha"].command("TransferAsset", src_account_id=user_a["id"], dest_account_id=user_b["id"],
                        asset_id=ASSET_ID, amount="100"),
    user_a["iroha"].command("TransferAsset", src_account_id=user_a["id"], dest_account_id=user_c["id"],
                        asset_id=ASSET_ID, amount="100")
]
tx = IrohaCrypto.sign_transaction(
    user_a["iroha"].transaction(command), user_a["private_key"])
print("Sending transaction...")
status = send_transaction(tx, net_1)
assert status[0] == "REJECTED"
print("Transaction rejected!")

# Now check that all parties have the correct asset total
for user in [user_a, user_b, user_c]:
    user_assets = get_user_assets(user['id'])
    assert str(user_assets) == f'[asset_id: "{ASSET_ID}"\naccount_id: "{user["id"]}"\nbalance: "100"\n]'
    print(user_assets)
printColor("DOUBLE SPEND PREVENTED")

So User A could not double spend their asset in the same transaction! What if instead of one transaction, User A tried sending the double spend in two transactions to two different nodes?

In [ ]:
set_user_asset_balance()

command1 = [
    user_a["iroha"].command("TransferAsset", src_account_id=user_a["id"], dest_account_id=user_b["id"],
                        asset_id=ASSET_ID, amount="100")
]

command2 = [
    user_a["iroha"].command("TransferAsset", src_account_id=user_a["id"], dest_account_id=user_c["id"],
                        asset_id=ASSET_ID, amount="100")
]
tx_1 = IrohaCrypto.sign_transaction(
    user_a["iroha"].transaction(command1), user_a["private_key"])
tx_2 = IrohaCrypto.sign_transaction(
    user_a["iroha"].transaction(command2), user_a["private_key"])

# We cannot use IrohaUtils.send_transaction as the send_transaction method 
# is blocking until a final status is found
# Unless swapping to threading or asyncio this means there will be some code duplication here
hex_hash_1 = binascii.hexlify(IrohaCrypto.hash(tx_1))
hex_hash_2 = binascii.hexlify(IrohaCrypto.hash(tx_2))
# Actually send the transactions, one after another
print("Sending transactions...")
net_1.send_tx(tx_1)
net_2.send_tx(tx_2)
# Get the status's of each transactions
last_status = [None, None]
for status in net_1.tx_status_stream(tx_1):
    last_status[0] = status
for status in net_2.tx_status_stream(tx_2):
    last_status[1] = status
# Sort the list of last status so we can check one commit and one reject
last_status.sort()
assert last_status[0][0]=="COMMITTED" and last_status[1][0]=="REJECTED"
print("One transaction accepted!")
print("One transaction rejected!")

# Now check that all parties have the correct asset total
for user in [user_a, user_b, user_c]:
    user_assets = get_user_assets(user['id'])
    print(user_assets)
printColor("DOUBLE SPEND PREVENTED")

The double spend was prevented, which is fantastic! Even using two different transactions with two different nodes over two different connections, the blockchain was able to reject one of the transactions and prevent double spending from occuring.

What if a malicious user tries more sinister methods of influencing the network? Maybe they could try to create a new role...

In [ ]:
print("Attempting to create a new role without permission")
commands = [
    # A new user that can add asset quantities, which is BAD 
    user_a["iroha"].command("CreateRole", role_name="new_user", permissions=[
        primitive_pb2.can_receive,
        primitive_pb2.can_transfer,
        primitive_pb2.can_add_asset_qty
    ])
]

tx = IrohaCrypto.sign_transaction(
    user_a["iroha"].transaction(commands), user_a["private_key"])
print("Sending transaction...")
status = send_transaction(tx, net_1)
assert status[0] == "REJECTED"
print("Transaction rejected!")
printColor("NO ROLE CREATED")

So a malicious user without permissions can not create a new role, what about a new user?

In [ ]:
print("Attempting to create a new account without permission")
user_x = new_user("user_x", DOMAIN_NAME)
commands = [
    user_a["iroha"].command('CreateAccount', account_name=f'{user_x["name"]}', domain_id=DOMAIN_NAME,
                      public_key=user_x["public_key"]),

]

tx = IrohaCrypto.sign_transaction(
    user_a["iroha"].transaction(commands), user_a["private_key"])
print("Sending transaction...")
status = send_transaction(tx, net_1)
assert status[0] == "REJECTED"
print("Transaction rejected!")
printColor("NO ACCOUNT CREATED")

So a malicious user cannot create a new account, locking off a lot of fraud options too!

Okay, what if a malicious user tried to hijack another users assets by sigining a fraudulent transaction?

In [ ]:
print("Attempting to sign as other user")
# Create a transfer from user c to user a of 10 asset
commands = [
    user_a["iroha"].command("TransferAsset", src_account_id=f'{user_c["id"]}', dest_account_id=f'{user_a["id"]}',
                asset_id=ASSET_ID, amount="10")
]

# Sign with own private key
tx = IrohaCrypto.sign_transaction(
    user_a["iroha"].transaction(commands), user_a["private_key"])

print("Sending transaction...")
status = send_transaction(tx, net_1)
assert status[0] == "REJECTED"
print("Transaction rejected!")
printColor("TRANSFER FAILED SUCCESSFULLY ;)")

Without knowing User C's private key, User A cannot impersonate them and steal their asset. What if User A *did* have User C's private key?

In [ ]:
set_user_asset_balance()
print("Attempting to sign with other users key")

# Create a transfer from user c to user a of 10 asset
commands = [
    user_a["iroha"].command("TransferAsset", src_account_id=f'{user_c["id"]}', dest_account_id=f'{user_a["id"]}',
                asset_id=ASSET_ID, amount="10")
]

# If User A knows only the private key, all other information can be derived
user_c_priv_key = user_c["private_key"]
user_c_iroha = Iroha(user_c["id"])


# Sign with user C private key
tx = IrohaCrypto.sign_transaction(
    user_c_iroha.transaction(commands), user_c["private_key"])

print("Sending transaction...")
status = send_transaction(tx, net_1)
assert status[0] == "COMMITTED"
print("Transaction accepted!")
for user in [user_a, user_c]:
    user_assets = get_user_assets(user['id'])
    print(user_assets)
printColor("User C asset transfered!")

Oh dear, if a private key is compromised then there is no security. This is because the private key forms the basis of trust in these networks. When a private key is compromised there is no way to verify if transactions from that key are valid or not, leading to serious issues in a network. Even if blockchain technology is otherwise a perfect fit for veracity related projects, it would be foolish to ignore the reliance on secret keys. A social engineering attack to compromise a secret key may not be very eloquent, but the relative ease (compared to cracking a secret key or controlling the network nodes) and ability to compromise a user means there is a great risk in these networks that could easily be overlooked.

On to more cheerful matters, we will investigate replay attacks! User A will attempt to replay their own transaction:

In [ ]:
set_user_asset_balance()
commands = [
    user_a["iroha"].command("TransferAsset", src_account_id=f'{user_a["id"]}', dest_account_id=f'{user_c["id"]}',
                asset_id=ASSET_ID, amount="10")
]

tx = IrohaCrypto.sign_transaction(
    user_a["iroha"].transaction(commands), user_a["private_key"])

print("Sending transaction...")
status = send_transaction(tx, net_1)
assert status[0] == "COMMITTED"
print(status)
print("FIRST TRANSACTION SUCCESSFUL")
status = send_transaction(tx, net_1)
print(status)
print("SECOND TRANSACTION... SUCCESSFUL?")
# As it turns out, Iroha *will* accept a committed transaction again but returns the old response without replaying the effect 
user_a_assets = get_user_assets(user_a['id'])
user_c_assets = get_user_assets(user_c["id"])
assert str(user_a_assets) == f'[asset_id: "{ASSET_ID}"\naccount_id: "{user_a["id"]}"\nbalance: "90"\n]'
assert str(user_c_assets) == f'[asset_id: "{ASSET_ID}"\naccount_id: "{user_c["id"]}"\nbalance: "110"\n]'
for user in [user_a, user_c]:
    user_assets = get_user_assets(user['id'])
    print(user_assets)
print("ONLY ONE TRANSACTION COMMITTED")
printColor("REPLAY ATTACK PREVENTED")

So a user cannot replay their own transaction! Surprisingly (and annoyingly) Iroha does not reject the second transaction but returns the status of the *first* transaction again, making it look like the transaction was committed twice.

What if User A tries to replay the transaction of a *different* user?

In [ ]:
set_user_asset_balance()
print("ATTEMPTING REPLAY ATTACK OF OTHERS TRANSACTION")
# User C sends some coin to User B
commands = [
    user_c["iroha"].command("TransferAsset", src_account_id=f'{user_c["id"]}', dest_account_id=f'{user_b["id"]}',
                asset_id=ASSET_ID, amount="10")
]

tx = user_c["iroha"].transaction(commands)

signed_tx = IrohaCrypto.sign_transaction(
    tx, user_c["private_key"])

print("Sending transaction...")
status = send_transaction(signed_tx, net_1)
assert status[0] == "COMMITTED"
print("FIRST TRANSACTION COMMITTED")

print("USER A ATTEMPTING REPLAY")
status = send_transaction(signed_tx, net_1)
# Again, the response code is from the first transaction, so is "committed" but the effect takes hold once
for user in [user_a, user_b, user_c]:
    user_assets = get_user_assets(user['id'])
    print(user_assets)
printColor("REPLAY ATTACK PREVENTED")

So again, the replay returns a committed code but in reality this references the original transaction, and no replay takes place

---

### Conclusions

These tests have shown:
- The Iroha network is reachable from python and has full functionality
- The Iroha network is robust to several different attacks
- The greatest threat to a blockchain network is compromised secret keys

Overall, I hope the tests above have improved your understanding of how the Iroha network works, and how the python SDK interacts with it. While these tests are not overly exciting, they are important to show that blockchain technology is an excellent tool but not the be-all-and-end-all solution to veracity that some may claim. 